In [67]:
import pandas as pd
from os import path, curdir

ROOT = path.abspath(curdir)

datapath = path.join(ROOT, 'data', 'data.csv')
datapath2 = path.join(ROOT, 'data', 'calculations.csv')

df = pd.read_csv(datapath)
df2 = pd.read_csv(datapath2)
df = pd.merge(df, df2, how= 'left', on='State')
df = df.iloc[:,1:]
df = df.rename(columns={'RecidivismRate_x':'RecidivismRate'})
df=df.drop(columns=['Unnamed: 0_y','RecidivismRate_y'])
print(df.columns)

Index(['State', 'RecidivismRate', 'PrisonPopulation', 'ImprisonmentRate',
       'Population', 'qualityoflifetotalscore', 'QualityOfLifeAffordability',
       'QualityOfLifeEconomy', 'QualityOfLifeEducationAndHealth',
       'QualityOfLifeSafety',
       ...
       'PercentFemale', 'PercentAsian', 'PercentWhite', 'PercentBlack',
       'PercentNativeAmerican', 'PercentOfPopInPrison', 'TotalViolentCrimes',
       'TotalNonviolentCrimes', 'OtherCrimes', 'AverageQOLScore'],
      dtype='object', length=118)


In [68]:
from sklearn.model_selection import train_test_split
X = df[['PercentMale','PercentBlack','PercentWhite']]
Y = df['RecidivismRate']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, train_size=0.8)


In [29]:
from sklearn import linear_model as lm
model = lm.LinearRegression()
model.fit(X_train,Y_train)
model.score(X_test, Y_test)

-0.40686290090083

In [30]:
from sklearn import tree
treeModel = tree.DecisionTreeRegressor()
treeModel.fit(X_train,Y_train)
treeModel.score(X_test,Y_test)

-0.6076114207440828

In [31]:
from sklearn.ensemble import RandomForestRegressor
treeModel = RandomForestRegressor()
treeModel.fit(X_train, Y_train)
treeModel.score(X_test,Y_test)

-0.0850833976336638

In [32]:
model = lm.Lasso(alpha=0.5)
model.fit(X_train, Y_train)
model.score(X_test, Y_test)


-0.19288277784386065

In [72]:
from sklearn import linear_model as lm

df['Percent_in_Prison'] = 1.0 * df['PrisonPopulation']/df['Population']
X = df[['Percent_in_Prison']]
Y = df['RecidivismRate']
model = lm.LinearRegression()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, train_size=0.8)
model.fit(X_train,Y_train)
model.score(X_test, Y_test)

-0.07821388101341342

In [73]:
df['Percent_Male_Under_18'] = 1.0 *(df['Male_Age_18']+df['Male_Age_17']+df['Male_Age_16'])/df['Total']
X = df[['Percent_Male_Under_18']]
Y = df['RecidivismRate']
model = lm.LinearRegression()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, train_size=0.8)
model.fit(X_train,Y_train)
model.score(X_test, Y_test)

-0.18950466473876282